In [1]:
import torch
import matplotlib.pyplot as plt

In [2]:
X = torch.tensor([[0.11, 0.09], [0.01, 0.02], [0.98, 0.91],
              [0.12, 0.21], [0.98, 0.99], [0.85, 0.87],
              [0.03, 0.14], [0.55, 0.45], [0.49, 0.51], 
              [0.99, 0.01], [0.02, 0.89], [0.31, 0.47],
              [0.55, 0.29], [0.87, 0.76], [0.63, 0.24]], dtype=torch.float)
X = torch.column_stack((X, torch.ones(15))) 
y = torch.tensor([-0.8, -0.97, 0.89, -0.67, 0.97, 0.72,
              -0.83, 0.00, 0.00, 0.00, -0.09, -0.22, 
              -0.16, 0.63, 0.37], dtype=torch.float)

# Let us compute solution using pseudo inverse
solution_pseudo = torch.matmul(torch.matmul(
    torch.linalg.inv(torch.matmul(X.T, X)), X.T) , y)
print("Solution via pseudo inverse: {}".format(solution_pseudo)) 


y = y.reshape((-1, 1))

# Let us define the torch module
class LinearModel(torch.nn.Module):
    def __init__(self, num_features):
        super(LinearModel, self).__init__()
        self.w = torch.nn.Parameter(
            torch.randn(num_features, 1))
    
    def forward(self, X):
        """
        In the forward function we accept a Tensor of input data 
        and we must return a Tensor of output data. 
        We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        y_pred  = torch.mm(X, self.w) # Computes Xw
        return y_pred

num_unknowns = 3
model =  LinearModel(num_features=num_unknowns)
# Let us use  Pytorch MSE loss function
loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

# Train model iteratively
num_steps = 1000
for step in range(num_steps):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    if step % 100 == 0:
        print("Loss at step {}: {}".format(step, loss))
    
    # Zero the gradients before running the backward pass.
    optimizer.zero_grad()
    # Compute the gradients for this step
    loss.backward()
    # Gradient descent
    optimizer.step()

solution_gd = torch.squeeze(model.w.data)
print("The solution via gradient descent is {}".format(solution_gd))

assert torch.allclose(solution_pseudo, solution_gd)


Solution via pseudo inverse: tensor([ 1.0766,  0.8976, -0.9582])
Loss at step 0: 16.155038833618164
Loss at step 100: 0.238687664270401
Loss at step 200: 0.217646986246109
Loss at step 300: 0.21724411845207214
Loss at step 400: 0.2172362506389618
Loss at step 500: 0.21723619103431702
Loss at step 600: 0.21723611652851105
Loss at step 700: 0.21723619103431702
Loss at step 800: 0.21723619103431702
Loss at step 900: 0.21723619103431702
The solution via gradient descent is tensor([ 1.0766,  0.8976, -0.9582])
